In [75]:
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras as keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D , Dropout,Conv3D, MaxPooling3D, Input, Deconv3D, BatchNormalization, Activation, Reshape
from keras.utils import to_categorical
from keras import optimizers

# init_g = tf.global_variables_initializer()
# tf.tables_initializer()
# with tf.Session() as sess:
#     sess.run(init_g)
    
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [76]:
label = np.load('Y.npy')
X = np.load('X.npy')
X = X.reshape(50,256,512,1)
print(X.shape)

label = label.reshape(1,128,128,128,1)
Y = np.repeat(label, 30,axis =0 )
Y.shape

(50, 256, 512, 1)


(30, 128, 128, 128, 1)

In [77]:
x_train = X[:30]
x_train.shape
x_val = X[30:40]
x_val.shape
x_test = X[40:50]
x_test.shape

(10, 256, 512, 1)

In [96]:
filters_in = [32,64,96]
kernel_size = [16,36,40]
strides = (4,4,4)
def encoder2D(kernel_size, strides):
    inputs = Input(shape=(256,512,1))

    g1 = Conv2D(filters=filters_in[0], kernel_size=kernel_size[0])(inputs)
    g1 = MaxPooling2D(pool_size=8)(g1)
    g1 = Activation(activation='relu')(g1)

    g6 = Flatten()(g1)
     
    g8 = Dense(20,activation='sigmoid')(g6)
    g8 = Activation(activation='sigmoid')(g8) 
    
    g9 = Reshape((1, 1,1,20), input_shape=(20,))(g8)
    
    g5 = Deconv3D(filters=1, kernel_size=128)(g9)
    g5 = BatchNormalization()(g5)
    g5 = Activation(activation='sigmoid')(g5) 
    
    model = Model(inputs=inputs, outputs=g5)
    model.summary()

    return model 
encoder2D = encoder2D(kernel_size,strides)  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 512, 1)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 241, 497, 32)      8224      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 30, 62, 32)        0         
_________________________________________________________________
activation_7 (Activation)    (None, 30, 62, 32)        0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 59520)             0         
_________________________________________________________________
dense_31 (Dense)             (None, 20)                1190420   
_________________________________________________________________
activation_8 (Activation)    (None, 20)                0         
__________

In [97]:
def create_model():
  model = Sequential()
  model.add(encoder2D)
 # model.add(generator)
  model.summary()
  model.compile(optimizer = 'adam', loss=['binary_crossentropy'], metrics=['mse'])
  return model

model = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_3 (Model)              (None, 128, 128, 128, 1)  43141689  
Total params: 43,141,689
Trainable params: 43,141,687
Non-trainable params: 2
_________________________________________________________________


In [98]:

def generator(x_train,y,batch_size=2):
    idx=0
    while(True):
        batch = x_train[idx:idx+2]
        label = np.concatenate((y,y),axis=0)
        batch = batch.reshape(2,256,512,1)
        label = label.reshape(2,128,128,128,1)

        print(batch.shape, label.shape)
        yield (batch,label)


In [102]:
model.fit_generator(generator(x_train,label,2),verbose=2,steps_per_epoch=1,epochs =2)
# Z = Y.reshape(30,128*128*128)
# Z.shape

Epoch 1/2
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
(2, 256, 512, 1) (2, 128, 128, 128, 1)
 - 3s - loss: 0.6134 - mean_squared_error: 0.2135
Epoch 2/2
(2, 256, 512, 1) (2, 128, 128, 128, 1)
 - 3s - loss: 0.5491 - mean_squared_error: 0.1801


In [82]:
model.fit(x_train,Z,epochs=2,batch_size=1,verbose=2)

Epoch 1/2


FailedPreconditionError: Attempting to use uninitialized value dense_28/kernel
	 [[{{node dense_28/kernel/read}}]]